Let's see if we can change the value in setitem.

In [3]:
import llvmlite

import numba

llvm_type = llvmlite.ir.ArrayType(llvmlite.ir.IntType(64), 1)


class TestStructType(numba.types.Type):
    def __init__(self):
        super().__init__(name="TestStruct")


test_struct_type = TestStructType()


@numba.extending.register_model(TestStructType)
class TestStructModel(numba.extending.models.PrimitiveModel):
    def __init__(self, dmm, fe_type):
        super().__init__(dmm, fe_type, llvm_type)

        

@numba.extending.intrinsic
def create_test_struct(typingctx, int_t):

    sig = test_struct_type(int_t)

    def codegen(context, builder, sig, args):
        a = builder.load(builder.alloca(llvm_type))
        ret = builder.insert_value(a, args[0], 0)
        return ret

    return sig, codegen


@numba.extending.intrinsic
def convert_test_struct(typingctx, test_struct_t):

    sig = numba.types.int64(test_struct_t)

    def codegen(context, builder, sig, args):
        return builder.extract_value(args[0], 0)

    return sig, codegen

@numba.extending.infer_getattr
class _InnerTemplate(numba.typing.templates.AttributeTemplate):
    key = TestStructType

    def generic_resolve(self, val, attr):
        if attr == 'first':
            return numba.types.int64

@numba.extending.lower_setattr(TestStructType, 'first')
def inner_settattr_impl(context, builder, sig, args):
    target, value = args
    return builder.insert_value(target, value, 0)


@numba.njit
def first_test():
    a = create_test_struct(10)
    print(convert_test_struct(a))
    a.first = 123
    print(convert_test_struct(a))


first_test()

10
10


Hmm not working. let's see if we can modify tuples

In [4]:
@numba.njit
def next_test():
    t = (1, 2, 3)
    print(convert_test_struct(a))
    a.first = 123
    print(convert_test_struct(a))


first_test()

10
10


Oh that's silly, of course we can't. 

Instead, let's see if we use pointers as the default value and just use values on enter/exit (copy on every function call).

In [8]:
class TestStructType2(numba.types.Type):
    def __init__(self):
        super().__init__(name="TestStruct")


test_struct_type_2 = TestStructType2()

        
def as_ptr(builder, value):
    new = builder.alloca(llvm_type)
    builder.store(new, value)
    return new
        

@numba.extending.register_model(TestStructType2)
class TestStructModel2(numba.extending.models.PrimitiveModel):
    def __init__(self, dmm, fe_type):
        super().__init__(dmm, fe_type, llvmlite.ir.PointerType(llvm_type))

#     def get_argument_type(self):
#         return ptr_llvm_type

    def get_return_type(self):
        return llvm_type

#     def get_data_type(self):
#         return ptr_llvm_type

#     def as_argument(self, builder, value):
#         return as_ptr(builder, value)

    def as_return(self, builder, value):
        return builder.load(value)

#     def as_data(self, builder, value):
#         return as_ptr(builder, value)

#     def from_argument(self, builder, value):
#         return builder.load(value)

    def from_return(self, builder, value):
        return as_ptr(builder, value)

#     def from_data(self, builder, value):
#         return builder.load(value)


    
@numba.extending.intrinsic
def create_test_struct2(typingctx, int_t):

    sig = test_struct_type_2(int_t)

    def codegen(context, builder, sig, args):
        a = builder.alloca(llvm_type)
        ret = builder.insert_value(builder.load(a), args[0], 0)
        builder.store(ret, a)
        return a

    return sig, codegen


@numba.extending.intrinsic
def convert_test_struct2(typingctx, test_struct_t):

    sig = numba.types.int64(test_struct_t)

    def codegen(context, builder, sig, args):
        return builder.extract_value(builder.load(args[0]), 0)

    return sig, codegen

@numba.extending.infer_getattr
class _InnerTemplate(numba.typing.templates.AttributeTemplate):
    key = TestStructType2

    def generic_resolve(self, val, attr):
        if attr == 'first':
            return numba.types.int64

@numba.extending.lower_setattr(TestStructType2, 'first')
def inner_settattr_impl(context, builder, sig, args):
    target, value = args
    ret = builder.insert_value(builder.load(target), value, 0)
    builder.store(ret, target)


@numba.njit
def first_test2():
    a = create_test_struct2(10)
    print(convert_test_struct2(a))
    a.first = 123
    print(convert_test_struct2(a))


first_test2()

10
123


Wow that works! Now let's go back to original bug report and try this out.